In [ ]:
from skimage import io
my_img = io.imread(r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images\ffba62c94141.jpg')
io.imshow(my_img)
io.show()

In [ ]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd

bsize=256

img_names = os.listdir(path)
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.1, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)
for idx, name in enumerate(img_names):
    if (i==bsize):
        break
    img_name = path + '\\' + name
    json_name = json_path + '\\' + name
    json_name = json_name.replace(json_name[len(json_name) - 3:], 'json')
    # Use you favourite library to load the image
    image = plt.imread(img_name)
    image = image.copy()
    image = torch.from_numpy(image)
    image = torch.unsqueeze(image, dim=0)
    image = torch.movedim(image, source=3, destination=1)
    image = functional.interpolate(image, size = (256,256))
    
    f = open(json_name)
    data = json.load(f)
    t = data['chart-type']
    images[idx] = image
    labels[idx] = t
    f.close()
    
    #image_resized = functional.interpolate(image, size = (128,128))
    i+=1

In [ ]:
images.shape

In [ ]:
#my_img=images[45]
io.imshow(my_img)
io.show()

In [ ]:
#labels

In [ ]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)

In [ ]:
## Loop through each parameter and set `requires_grad` to false
for param in efnet_model.features.parameters():
    param.requires_grad = False

In [ ]:
## Replace the "classifier" layer with one for our application
efnet_model.classifier = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1280, out_features=4, bias=True))

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

train_y=labels
train_X=images

encode_list = [b'vertical_bar',b'scatter',b'dot',b'line']
# Create an instance of the OneHotEncoder class and fit it on the data
encoder = OneHotEncoder(sparse=False)
encoder.fit(train_y.reshape(-1, 1))

# Transform the data using the fitted encoder
encoded_data = encoder.transform(train_y.reshape(-1, 1))

# Map the encoded data to the desired format
train_y = [encoder.categories_[0].tolist().index(category) for category in encoder.inverse_transform(encoded_data).flatten()]

print(train_y)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

y_tensor = torch.tensor(train_y)

### Convert to numpy array then reshape to 900 by 28 by 28
mnist_unflattened = train_X
mnist_unflattened = mnist_unflattened.reshape(256, 3, 256, 256)

## Convert to tensor
mnist_tensor = torch.from_numpy(mnist_unflattened)
mnist_tensor =  torch.unsqueeze(mnist_tensor, dim=1)

## Transform to proper input shape
new_mnist_tensors = mnist_tensor.expand(-1, -1, -1, -1, -1)

train_loader = DataLoader(TensorDataset(new_mnist_tensors.type(torch.FloatTensor), y_tensor.type(torch.LongTensor)), batch_size=32)

In [ ]:
## Hyperparms
epochs = 100
lrate = 0.1

## Cost Function
from torch import nn
cost_fn = nn.NLLLoss()

## Network model
torch.manual_seed(7) # For reproduction purposes (should be minor since only the last layers are randomly intialized)
net = efnet_model

## Optimizer (using ADAM, a more flexible algorithm than SGD this time)
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)

## Initial values for cost tracking
track_cost = np.zeros(epochs)
cur_cost = 0.0

## Loop through the data
for epoch in range(epochs):
    
    cur_cost = 0.0
    correct = 0.0
    
    ## train_loader is iterable and numbers knows the batch
    for i, data in enumerate(train_loader, 0):
        
        ## The input tensor and labels tensor for the current batch
        inputs, labels = data
        ## Clear the gradient from the previous batch
        optimizer.zero_grad()
        
        ## Provide the input tensor into the network to get outputs
        inputs = inputs.reshape((32, 3, 256, 256))
        outputs = net(inputs)
        
        ## Calculate the cost for the current batch
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        
        ## Calculate the gradient
        cost.backward()
        
        ## Update the model parameters using the gradient
        optimizer.step()
        
        ## Track the current cost (accumulating across batches)
        cur_cost += cost.item()
    
    ## Store the accumulated cost at each epoch
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}") ## Uncomment this if you want printed updates


In [ ]:
#Take 2

In [1]:
path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\images'
json_path = r'C:\Users\16032\Downloads\benetech-making-graphs-accessible\train\annotations'
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional
import json
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OneHotEncoder
from pandas import get_dummies

bsize=256

img_names = os.listdir(path)
#img_name = path + '\\' + name
#json_names = os.listdir(json_path)
i=0
#ADD TRAIN TEST SPLIT HERE TO IMG_NAMES
from sklearn.model_selection import train_test_split
train, test = train_test_split(img_names, test_size=0.1, random_state=5)

images = np.empty(shape = (bsize, 3, 256, 256))
labels = np.chararray(bsize, itemsize=20)

In [2]:
train_y = train.copy()
train_X = train.copy()
for i in range(len(train)):
    train_X[i] = path + '\\' + img_names[i]
    train_y[i] = json_path + '\\' + img_names[i]
    train_y[i] = train_y[i].replace(train_y[i][len(train_y[i]) - 3:], 'json')

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [4]:
class MyDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        f = open(self.label_paths[index])
        data = json.load(f)
        t = data['chart-type']
        #print(t)
        if (t=='scatter'):
            label = 0
        elif (t=='vertical_bar'):
            label = 1
        elif (t=='dot'):
            label = 2
        else:
            label = 3
        f.close()
        
        if self.transform:
            image = self.transform(image)
            #label = self.transform(label)
            
        return image, label

# Define the transform to resize and normalize the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create a dataset from the image and label file paths
dataset = MyDataset(train_X, train_y, transform=transform)

# Create a dataloader to load the data in batches
batch_size = 256
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)
## Loop through each parameter and set `requires_grad` to false
for param in efnet_model.features.parameters():
    param.requires_grad = False

## Replace the "classifier" layer with one for our application
efnet_model.classifier = torch.nn.Sequential( 
    torch.nn.Linear(in_features=1280, out_features=4, bias=True))

In [ ]:
encoder = OneHotEncoder(sparse=False)

# Train the neural network on the data using batches
from torch import nn
epochs = 100
lrate = 0.1
cost_fn = nn.NLLLoss()
net = efnet_model
optimizer = torch.optim.Adam(net.parameters(), lr=lrate)
track_cost = np.zeros(epochs)

for epoch in range(epochs):
    cur_cost = 0.0
    correct = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs = inputs.reshape(inputs.size())
        outputs = net(inputs)
        #print(labels)
        
        cost = cost_fn(outputs.log_softmax(dim=1), labels)
        cost.backward()
        optimizer.step()
        cur_cost += cost.item()
        
    track_cost[epoch] = cur_cost
    print(f"Epoch: {epoch} Cost: {cur_cost}")

Epoch: 0 Cost: 43.08404497546144
Epoch: 1 Cost: 16.833578320409288


In [ ]:
inputs.size()